# Create Daily Heists

In [1]:
import os, sys, random, json
import pandas as PD

In [2]:
BACKEND_PATH = os.path.dirname(os.path.dirname(os.getcwd()))
APPPROJ_PATH = os.path.join(BACKEND_PATH, 'app_proj')
sys.path.append(APPPROJ_PATH)
import notebooks as NT
print([x for x in dir(NT) if x[0]!='_' and len(x)>8])

['DataframeToDicts', 'GetRandom', 'NOTEBOOK_ENV']


In [3]:
import django
os.environ['DEPLOYMENT_ENV'] = NT.NOTEBOOK_ENV
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'True'
MODULE_PATH = os.path.dirname(os.getcwd())
os.chdir(os.path.dirname(MODULE_PATH))
django.setup()

env: DEV


In [4]:
import database as DB
print([x for x in dir(DB) if x[0]!='_' and len(x)>8])

['BaseManager', 'DeleteTable', 'GetNativeTableCounts', 'GetTableCounts', 'GetTableDictionary', 'InsertBulk', 'InsertSingle']


In [5]:
import django.db.models as JM
from django.utils import timezone
from django.contrib.auth import get_user_model

In [6]:
import emporium.models as EM
import emporium.logic.guild as GD
import emporium.logic.stage as ST
import emporium.logic.simulation as SM
print([x for x in dir(EM) if x[0]!='_' and len(x)>8])

['FactoryUpgrades', 'GothicTower', 'KeepUpgrades', 'LeagueTrial', 'SingletonUpgrades', 'StorageUpgrades', 'ThiefLevel', 'UnlockableItem', 'UnlockableThief']


In [7]:
import engine.models as GM
import engine.logic.content as CT
import engine.logic.launcher as LH
import engine.logic.resource as RS
import engine.logic.robot as RB
print([x for x in dir(GM) if x[0]!='_' and len(x)>8])

['GuildStage', 'ItemInGuild', 'ItemUnlocked', 'ThiefInGuild', 'ThiefUnlocked']


In [8]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

### Daily Heists

In [9]:
# get user and guild

userMd = get_user_model().objects.filter(user_name='Admin')[0]
userMd

<User: admin@thieves-guild.com>

In [10]:
guildMd = GM.Guild.objects.GetOrNone(UserFK=userMd, Selected=True)

if not guildMd:
    responseDx = {
        'thiefLs': None,
        'assetLs': None,
        'message': '* A guild must be chosen in the Account page.',
    }

guildMd

<Guild: Guild object (10)>

In [11]:
currDt = timezone.now()
currMonth = currDt.month
currDay = currDt.day
currWeekDay = currDt.weekday()   # 0 monday, 1 tuesday, .. 6 sunday
currDate = f"{currDt.year}-{str(currMonth).zfill(2)}-{str(currDay).zfill(2)}"

print('month:', currMonth, '| day:', currDay, '| week day:', currWeekDay, '| full: ', currDate)

month: 2 | day: 12 | week day: 0 | full:  2024-02-12


### Guild Tower

In [12]:
# check for existing daily stages

checkStages = GM.GuildStage.objects.filter(
    GuildFK=guildMd, Heist='tower', CreateDate=currDate
    ).values()

if checkStages:
    stageDf = PD.DataFrame(checkStages).drop(['_state', 'GuildFK_id'], axis=1, errors='ignore')
    stageDf = stageDf.drop_duplicates(subset=['StageNo']).sort_values('StageNo')
    stageLs = NT.DataframeToDicts(stageDf)
    PD.DataFrame(stageLs)[:3]

else:
    print('no stages for', currDate)

,id,Heist,StageNo,CreateDate,RoomTypes,CompleteRooms,Background,BackgroundRoomBias,ObstaclesR1,ObstaclesR2,ObstaclesR3,ObstaclesR4,ObstaclesR5
0,1441,tower,1,2024-02-12,"[balanced, None]","[False, None]",temple,"[0, None]","[{""Level"": 1, ""Name"": ""Sorcerer"", ""Trait"": ""Al...",None,None,None,None
1,1442,tower,2,2024-02-12,"[balanced, None]","[False, None]",warehouse,"[0, None]","[{""Level"": 1, ""Name"": ""Crossbow Trap"", ""Trait""...",None,None,None,None
2,1443,tower,3,2024-02-12,"[balanced, None]","[False, None]",college,"[1, None]","[{""Level"": 1, ""Name"": ""Warden"", ""Trait"": ""All""...",None,None,None,None


In [13]:
# create when daily update is needed

GM.GuildStage.objects.filter(GuildFK=guildMd, Heist='tower').delete()

rawStages = list(EM.GothicTower.objects.filter(Keep=guildMd.KeepLevel).values())
lastType = ''
lastBackground = ''

for st in rawStages:

    newStage = GM.GuildStage()
    newStage.GuildFK = guildMd
    newStage.Heist = 'tower'
    newStage.StageNo = st['StageNo']
    newStage.CreateDate = currDate
    newStage.RoomTypes = []
    newStage.CompleteRooms = []
    newStage.BackgroundRoomBias = []
    background = ST.StageBackground(lastBackground)
    newStage.Background = background
    lastBackground = background

    # room 1

    roomType = ST.RandomRoomType(lastType)
    lastType = roomType
    obstacles = ST.AssembleRoom(roomType, st['LevelR1'], st['ObstaclesR1'])
    newStage.RoomTypes.append(roomType)
    newStage.CompleteRooms.append(False)
    newStage.BackgroundRoomBias.append(random.randint(0,1))
    newStage.ObstaclesR1 = obstacles

    # room 2

    if st['LevelR2']:
        roomType = ST.RandomRoomType(lastType)
        lastType = roomType
        obstacles = ST.AssembleRoom(roomType, st['LevelR2'], st['ObstaclesR2'])
        newStage.RoomTypes.append(roomType)
        newStage.CompleteRooms.append(False)
        newStage.BackgroundRoomBias.append(random.randint(0,1))
        newStage.ObstaclesR2 = obstacles
    else:
        newStage.RoomTypes.append(None)
        newStage.CompleteRooms.append(None)
        newStage.BackgroundRoomBias.append(None)

    newStage.save()

(12, {'engine.GuildStage': 12})

In [14]:
stageLs = GM.GuildStage.objects.filter(GuildFK=guildMd, Heist='tower').values()
stageDf = PD.DataFrame(stageLs).drop(['_state', 'GuildFK_id'], axis=1, errors='ignore')
stageDf = stageDf.drop_duplicates(subset=['StageNo']).sort_values('StageNo')
stageLs = NT.DataframeToDicts(stageDf)
PD.DataFrame(stageLs)[2:8]

,id,Heist,StageNo,CreateDate,RoomTypes,CompleteRooms,Background,BackgroundRoomBias,ObstaclesR1,ObstaclesR2,ObstaclesR3,ObstaclesR4,ObstaclesR5
2,1455,tower,3,2024-02-12,"[balanced, None]","[False, None]",college,"[1, None]","[{'Level': 1, 'Name': 'Door', 'Trait': 'Agi', ...",None,None,None,None
3,1456,tower,4,2024-02-12,"[balanced, None]","[False, None]",nobleman,"[1, None]","[{'Level': 1, 'Name': 'Balcony', 'Trait': 'Agi...",None,None,None,None
4,1457,tower,5,2024-02-12,"[balanced, None]","[False, None]",warehouse,"[1, None]","[{'Level': 1, 'Name': 'Door', 'Trait': 'Agi', ...",None,None,None,None
5,1458,tower,6,2024-02-12,"[balanced, biased mig]","[False, False]",temple,"[1, 0]","[{'Level': 2, 'Name': 'Sorcerer', 'Trait': 'Al...","[{""Level"": 1, ""Name"": ""Gargoyle"", ""Trait"": ""Mi...",None,None,None
6,1459,tower,7,2024-02-12,"[balanced, balanced]","[False, False]",nobleman,"[0, 0]","[{'Level': 1, 'Name': 'Arcane Seal', 'Trait': ...","[{""Level"": 1, ""Name"": ""Vanguard"", ""Trait"": ""Al...",None,None,None
7,1460,tower,8,2024-02-12,"[biased cmb, balanced]","[False, False]",college,"[1, 0]","[{'Level': 1, 'Name': 'Vanguard', 'Trait': 'Al...","[{""Level"": 2, ""Name"": ""Sewer Grate"", ""Trait"": ...",None,None,None


In [16]:
# clean up

GM.GuildStage.objects.filter(GuildFK=guildMd, Heist='tower').delete()

(0, {})